# ERA5 temperature timeseries: visualization

In this tutorial, we will visualise historical data from ERA5, a global reanalysis product.

## Preliminaries
This tutorial requires the presence of various Python libraries that may need to be installed. If you know that these libraries are already installed, you may skip this step.

In [ ]:
%%capture
!pip install xarray
!pip install matplotlib
!pip install numpy
!pip install cartopy

Once installed, the various packages need to be loaded into memory.

In [ ]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

## Location of the data
This tutorial assumes that data has already been downloaded. Using `xarray`, the dataset is accessed: 

In [ ]:
era5_data = xr.open_dataset(f'./era5_temperature_pays_bas_2021_2022_2023.nc') - 273.15
era5_data

## Plotting function
Below function is used to plot our data. Note that this function is taken from a script that was written by ECMWF (https://ecmwf-projects.github.io/copernicus-training-c3s/glofas-bangladesh-floods.html#explore-and-view-the-data).

In below function, the extent has the convention `[x0,x1,y0,y1]`. This convention differs from the one needed to download data from the CDS!

In [ ]:
# Create a simple plotting function that we can use throughout this notebook
def plot_map(
    plot_data,
    title='',
    cbar_label='',
    cmap='PuBu',
    extent=[-180, 180, -90, 90], #[x0,x1,y0,y1] = [West,East,South,North]
    **pcolorkwargs
):
    # Populate the title and cbar_label with attributes from the plot_data if they have not been
    #  explicitly specified
    title = title or plot_data.attrs.get('long_name', title)
    cbar_label = cbar_label or plot_data.attrs.get('units', cbar_label)

    # Create a figure with a cartopy projection assigned which allows plotting geospatial data
    fig, ax = plt.subplots(
        1, 1, figsize = (18, 9), subplot_kw={'projection': ccrs.PlateCarree()}
    )

    # Plot the data on our figure
    im = ax.pcolormesh(
        plot_data.longitude, plot_data.latitude, plot_data, cmap=cmap, **pcolorkwargs
    )

    # Add some additional features
    ax.set_title(title, fontsize=16)
    ax.gridlines(draw_labels=False, linewidth=1, color='gray', alpha=0.5, linestyle='--') 
    ax.coastlines(color='black')

    # Add country borders in red
    ax.add_feature(cfeature.BORDERS, edgecolor='black', lw=1.5, ls=":")

    # Set the plot domain/extent
    ax.set_extent(extent, crs=ccrs.PlateCarree())

    # Add a colour bar
    cbar = plt.colorbar(im,fraction=0.04, pad=0.01)
    cbar.set_label(cbar_label, fontsize=12) 

## Plot mean temperature
First, the average streamflow is plotted. This is an average over all data that is present in the file we accessed.

In [ ]:
mean_data = era5_data.t2m.mean(dim='time')
mean_data = mean_data.assign_attrs(**era5_data.t2m.attrs)
plot_map(
    mean_data,
    vmax=30, vmin=-10,
    cbar_label = "oC",
    extent=[3.5, 7.5, 50.5, 54], #[x0,x1,y0,y1] = [West,East,South,North]
    title = 'Mean temperature in The Netherlands, 2021-2023',
    cmap = 'coolwarm'
)

Let's now plot the data for a given moment:

In [ ]:
my_data = era5_data.t2m.sel(time='2021-07-23T14:00:00')
my_data = my_data.assign_attrs(**era5_data.t2m.attrs)
plot_map(
    my_data,
    vmax=30, vmin=-10,
    cbar_label = "oC",
    extent=[3.5, 7.5, 50, 54], #[x0,x1,y0,y1] = [West,East,South,North]
    title = 'Temperature in The Netherlands on a specific time',
    cmap = 'coolwarm'
)

## Plot data for a single location
The data file contains gridded data. From the grid, we extract data for a single location. We then plot the full timeseries.

In [ ]:
era5_data.t2m.sel(latitude=52, longitude=4.75, method='nearest').plot()